# 26 Point 2

## Step 0: Utils for Jupyter Notebooks

### 1.1.1 Add Show/Hide code button

In [4]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [51]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate Models + Props

In [63]:
def export_csv(strat_1, strat_2):
    with open(f"26Point2/params/26Point2_{strat_1}_{strat_2}.csv", 'w') as f:
        f.write("players, 2\n")
        f.write("no_sides, 4\n")
        f.write("no_dice, 4\n")
        f.write("p1, 1\n")
        f.write("p2, 2\n")
        f.write(f"strategy1, {strat_1}\n")
        f.write(f"strategy2, {strat_2}\n")
    return f"26Point2/params/26Point2_{strat_1}_{strat_2}.csv"

In [64]:
def build_model(strat_1, strat_2):
    param_file = export_csv(strat_1, strat_2)

    model_spec = f"26Point2_{strat_1}_{strat_2}"

    !mkdir "26Point2/models"
    output_model = f"26Point2/models/{model_spec}.prism"

    !python utils/pyprism.py 26Point2/26Point2.pyprism $output_model {param_file}

In [80]:
from IPython.display import Javascript


strats = [[1, 1], [2,1], [3,1], [4,1], [4,2], [5,1], [5,2], [5,3], [5,4]]

for strat in strats:
    strat_1, strat_2 = strat
    build_model(strat_1, strat_2)

A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_1_1.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_2_1.prism


A subdirectory or file 26Point2/models already exists.


...

Model generated successfully at 26Point2\models\26Point2_5_2.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_5_3.prism


A subdirectory or file 26Point2/models already exists.


Model generated successfully at 26Point2\models\26Point2_5_4.prism


# Properties to consider

* Win rate with various strategies (pick 2, pick 3, pick 4, pick 3/4)

* Adversary to pick 2/pick 4, pick3/4) (figure out how long strategy generation takes...)

* Potentially figure out bust rate? Though it may be nicer to do this theoretically for non-hybrid strategies...

In [81]:
strategies = []
# strats have (model_spec, no_spaces, prop_number, log_filename)

strategies.append(["1_1", 50, 1, "random_winrate_50.log"]) # random strat, should be 50% winrate (to convergence)
strategies.append(["4_2", 50, 1, "winrate_4_against_2.log"]) # risky vs safe strats - does the higher average compensate for higher bustrate?
strategies.append(["5_2", 50, 1, "hybrid_against_2.log"]) # safest strategy vs hybrid strategy
strategies.append(["5_3", 50, 1, "hybrid_against_3.log"]) # hybrid vs three
strategies.append(["5_4", 50, 1, "hybrid_against_4.log"]) # hybrid vs four


## Step 2a: Make auto.sh file

The big challenge with this case study is that the models are far too large to examine normally. Instead, we create a shell file that can run on a computing cluster.

In [82]:
def make_shell_file(filename, strats):
    with open(filename, 'w') as shell:
        for strat in strats:
            shell.write(f"{get_command(strat)}\n\n")

def get_command(strategy):
    model_spec, no_spaces, prop, log_file = strategy
    model = f"26Point2_{model_spec}"
    
    # will need to improve later for adversary, but fine for now
    command = f"~/prism-games-csg-dev/prism/bin/prism {model}.prism 26Point2_manual.props -javamaxmem 700g -const no_spaces={no_spaces} -prop {prop} > cluster/{log_file}"
    
    return(command)

In [83]:
make_shell_file("26Point2/logs/auto_generated.sh", strategies)

## Step 2b: Make logs with statistical model checking.

Statistical model checking isn't great, but it's far quicker than probabilistic model checking so it's appropriate for "quick checking" purposes.

In [84]:
def get_smc_logs(strats):
    for strat in strats:
        !{get_smc(strat)}

def get_smc(strategy):
    model_spec, no_spaces, prop, log_file = strategy
    model = f"26Point2_{model_spec}"
    
    # will need to improve later for adversary, but fine for now
    command = f"prism 26Point2/models/{model}.prism 26Point2/props/26Point2_manual.props -const no_spaces={no_spaces} -prop {prop} -sim -simmethod ci -simsamples 10000 > 26Point2/logs/smc/{log_file}"
    
    return(command)

In [85]:
get_smc_logs(strategies)

In [74]:
!prism 26Point2/models/26Point2_1_1.prism 26Point2/props/26Point2_manual.props  -const no_spaces=50 -prop 1 -sim -simmethod apmc -simsamples 10000 > 26Point2/logs/smc/autogen.log

In [24]:
!prism 26Point2/models/{model_spec}.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 6 -sim -simmethod ci -simsamples 10000

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Mon Mar 01 14:52:26 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games 26Point2/models/26Point2_4_1.prism 26Point2/props/26Point2_manual.props -const no_spaces=100 -prop 6 -sim -simmethod ci -simsamples 10000

Parsing model file "26Point2/models/26Point2_4_1.prism"...

Type:        CSG
Modules:     player1 player2 board 
Variables:   p1_pos p1_chosen_dice p1_1s p1_2s p1_3s p1_4s p2_pos p2_chosen_dice p2_1s p2_2s p2_3s p2_4s state winner 

Switching to explicit engine, which supports CSGs...

Parsing properties file "26Point2/props/26Point2_manual.props"...

7 properties:
(1) Pmax=? [ F "p1_wins" ]
(2) Pmax=? [ F "p2_wins" ]
(3) Pmax=? [ F "players_tie" ]
(4) Pmax=? [ F "game_over" ]
(5) R{"no_rounds"}max=? [ F "game_over" ]
(6) R{"p1_busts"}max=? [ F "game_over" ]
(7) R{"p1_busts"}max=? [ F "game_over" ]/R{"no_rounds"}max=? [ F "game_over" ]

------------------------------------------

## Step 3: Store data

Once we have the log files from the cluster, we then filter those files and generate more readable data. (Note that we perform this filtering at this stage, not on the cluster - that way, we might not need to run everything again if we need more data than we anticipated.